# 네이버 매장 -> 매장에 있는 tag 

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

from bs4 import BeautifulSoup

import datetime
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
res_df['검색어'] = res_df['상세주소'] + " " + res_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
res_df = res_df[column_order]
res_df.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [6]:
res_df.to_excel('./data/정리된매장df.xlsx')

### 테그 데이터 크롤링

In [7]:
res_df= res_df[:100]

In [8]:
# 해민
# driver = webdriver.Chrome()

# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())

# 빈 리스트 생성
restaurant_name_list = []
category_name_list = []
price_list = []


for i in range(0,len(res_df)):
  
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href

        driver.get(review_url)
        time.sleep(2.2)  

        for _ in range(4):
        # 스크롤을 위한 JavaScript 코드 실행
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

        # HTML 추출
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

        # # tag 끝까지 나오게 클릭 
        # try :
        #     while driver.find_elements(By.CLASS_NAME, 'Tvx37'):
        #         try:
        #             button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]')
        #             button.click()
        #             time.sleep(0.4)
        #         except ElementClickInterceptedException:
        #             break
        # except NoSuchElementException:
        #     print("No more '더보기' button found. Exiting the loop.")

        # 여러 식당 정보를 반복하며 리스트에 추가
        for item in items:
            restaurant_name_list.append(name)
            category_name_list.append(item.text.split('"')[1])
            price_list.append(int(''.join(filter(str.isdigit, item.text))))

        data_list = {
            '매장명': restaurant_name_list,
            'tag': category_name_list,
            '인원': price_list
        }
    except :
        data_list = {
            '매장명': restaurant_name_list,
            'tag': '리뷰 10개 미만',
            '인원': ''
        }
        continue

result_df = pd.DataFrame(data_list)
result_df

KeyboardInterrupt: 

In [ ]:
result_df.to_csv('./data/음식점_태그_샘플100개.csv')

### 리뷰 데이터 크롤링

In [13]:
# 일부 테스트
res_df_test = res_df.copy()
new_res_df_test = res_df_test.iloc[:10].copy()
new_res_df_test.reset_index(drop=True, inplace=True)
new_res_df_test

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"강남대로114길 18, 정식당",한식,정식당,정식당,서울특별시,강남구,"강남대로114길 18,",서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",202203.971845,444853.761458,가능,영업/정상
1,"강남대로 378, 채선당 샤브보트",한식,채선당 샤브보트,채선당 샤브보트 강남역점,서울특별시,강남구,"강남대로 378,",서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",202462.794327,443827.961945,가능,영업/정상
2,"압구정로 343, 수아당",한식,수아당,수아당,서울특별시,강남구,"압구정로 343,",서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",203470.848439,447369.579852,가능,영업/정상
3,"역삼로 204, 알찬한끼",분식,알찬한끼,알찬한끼,서울특별시,강남구,"역삼로 204,",서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",203453.074369,443736.685792,가능,영업/정상
4,"삼성로96길 11, 세계관",한식,세계관,세계관,서울특별시,강남구,"삼성로96길 11,",서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",204952.006872,445195.026116,가능,영업/정상
5,"역삼로 409, 분식을 품다",분식,분식을 품다,분식을 품다,서울특별시,강남구,"역삼로 409,",서울특별시 강남구 대치동 907-12 아티스톤,"서울특별시 강남구 역삼로 409, 지상1층 102호 (대치동, 아티스톤)",204506.415174,444321.238711,가능,영업/정상
6,"논현로146길 11, 식당 민홍",일식,식당 민홍,식당 민홍,서울특별시,강남구,"논현로146길 11,",서울특별시 강남구 논현동 66-35,"서울특별시 강남구 논현로146길 11, 지상2층 (논현동)",202574.840575,446222.990406,가능,영업/정상
7,"논현로157길 55, 디어",기타,디어,디어(Dear),서울특별시,강남구,"논현로157길 55,",서울특별시 강남구 신사동 555,"서울특별시 강남구 논현로157길 55, 지상1층 103호 (신사동)",202072.552980,446507.442186,가능,영업/정상
8,"개포로 506, 버거리",기타,버거리,버거리 개포동점,서울특별시,강남구,"개포로 506,",서울특별시 강남구 개포동 186-14 청우빌딩,"서울특별시 강남구 개포로 506, 청우빌딩 지상1층 2호 (개포동)",205901.723630,442995.181704,가능,영업/정상
9,"논현로8길 11, 제주그집",한식,제주그집,제주그집,서울특별시,강남구,"논현로8길 11,",서울특별시 강남구 개포동 1187-11,"서울특별시 강남구 논현로8길 11, 지상1층 (개포동)",204337.295359,441316.681277,가능,영업/정상


# 리뷰 url로 넘어가는 파트 

# 내용더보기 <- 버튼 만들어야함

In [14]:
# 빈 리스트 생성
user_id_list = []
content_list = []
date_list = []  
revisit_list = []
tag_category_name_list = []
restaurant_name_list = []
review_category_name_list = []

# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())


for i in range(0,len(new_res_df_test)):
  
    name = new_res_df_test['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href

        driver.get(review_url)
        time.sleep(2.2)  

    except :
        print('끝')


# ---------------- 여기가 수정 -------------------------
    # 현재 페이지 URL 가져오기
    current_url = driver.current_url

    try : 
        # 리뷰 url로 이동 
        modified_url = current_url.replace('/home', '/review/visitor')
        driver.get(modified_url)
    except :
        continue

    
# --------------------- 수정 중 ---------------------------------------------
    
    # 리뷰 스크래핑 시작
    try:
        # 스크롤
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 
            
            # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # 더보기 클릭
            button = driver.find_element_by_class_name('fvwqf')
            button.click()

            # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # -------------------------------------
            time.sleep(3)
            html = driver.page_source
            bs = BeautifulSoup(html, 'lxml')
            reviews = bs.select('li.YeINN')

        for r in reviews: 
            user_id = r.select_one('div.VYGLG')
            content = r.select_one('div.ZZ4OK.IwhtZ')
            try :
                tag = r.select_one('div.gyAGI span.P1zUJ').text
            except :
                tag = ""
            date = r.select('div._7kR3e>span.tzZTd>time')[0]
            revisit = r.select('div._7kR3e>span.tzZTd')[1]

            # 아무것도 없을 때
            user_id = user_id.text if user_id else ''
            content = content.text if content else ''
            tag = tag if tag else ''
            date = date.text if date else ''
            revisit = revisit.text if revisit else ''
            time.sleep(0.06)

            # Append data to lists
            restaurant_name_list.append(name)
            user_id_list.append(user_id)
            content_list.append(content)
            review_category_name_list.append(tag)
            date_list.append(date)
            revisit_list.append(revisit)    

    except Exception as e:
        print(e)

    # finally:
    #     driver.quit()

# Create a DataFrame
review_data_list = {
    '매장명': restaurant_name_list,
    'user_id': user_id_list,
    'content': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}

# 태그 데이터 프레임 완성
review_result_df = pd.DataFrame(review_data_list)

review_result_df


Message: 

Message: 

Message: element click intercepted: Element <span class="TeItc">...</span> is not clickable at point (531, 18). Other element would receive the click: <h1 id="_header" class="bh9OH" style="opacity: 1;">...</h1>
  (Session info: chrome=122.0.6261.69)

Message: 

Message: 

Message: 

Message: 

Message: 



,매장명,user_id,content,date,revisit,tag
0,"강남대로 378, 채선당 샤브보트",체리는 맛있어,,2.8.목,1번째 방문,친절해요
1,"강남대로 378, 채선당 샤브보트",하늘빛눈,"혼자 와서 어색했는데, 1인분 기준으로 만들어져서 진짜 편하고 배불리 먹다갑니다! ...",1.27.토,1번째 방문,매장이 넓어요
2,"강남대로 378, 채선당 샤브보트",Sso30,혼밥하기 너무 좋고 깔끔하고 청결하고 맛도 맛있어요~~ 고기 상태도 너무 괜찮고 야...,2.21.수,1번째 방문,음식이 맛있어요
3,"강남대로 378, 채선당 샤브보트",pim****,채선당 샤브보트 처음 먹어봤는데 재료 신선하고 구성도 좋고 너무 맛있네요! 앞으로 ...,2.6.화,1번째 방문,음식이 맛있어요
4,"강남대로 378, 채선당 샤브보트",신렐데라,"응대나 직원분 태도가 아쉽다는 이전 후기 보고 조금 걱정했는데, 사장님 완전 친절하...",2.24.토,1번째 방문,혼밥하기 좋아요
...,...,...,...,...,...,...
75,"삼성로96길 11, 세계관",나바래,맛도리 덮밥임. \n곱도리탕도 꿀맛인데 급하게 먹느라 사진 못찍음,23.12.17.일,1번째 방문,음식이 맛있어요
76,"삼성로96길 11, 세계관",나야나9247,투뿔한우곱도리탕~~ 칼칼하니 맛있어요 사리넣어서 먹기 좋습니다\n,1.19.금,1번째 방문,음식이 맛있어요
77,"삼성로96길 11, 세계관",감자깡2735,친구들이랑 지나가다가 못 보던 가게가 생겼길래 들어왔다가 단골되게 생겼어요!!!!!...,23.12.18.월,1번째 방문,음식이 맛있어요
78,"삼성로96길 11, 세계관",니뿡36,맛있고 친절해요,2.16.금,1번째 방문,음식이 맛있어요


In [15]:
review_data_list

{'매장명': ['강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '강남대로 378, 채선당 샤브보트',
  '삼성로96길 11, 세계관',
  '삼성로96길 11,

In [17]:
review_result_df.to_excel('리뷰확인.xlsx')

In [ ]:
# review_result_df.drop(review_result_df.index, inplace=True)
# review_result_df

,user_id,content,date,revisit,tag


In [ ]:
# 원본

url = 'https://m.place.naver.com/restaurant/13149660/review/visitor?entry=pll'

# 매장명
place = '강남역_서브웨이'

# 리뷰 스크래핑 시작
try:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    res = driver.get(url)
    driver.implicitly_wait(2)

    # 스크롤
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 
        
        # '더보기' 클릭
        more_button = driver.find_element(By.XPATH, '//div[@class="lfH3O"]/a[@role="button"]')
        more_button.click()

        # 이걸 하니까 적당한 사람 myplace url 생김. 상단 3명 url 가져오라고 할까?
        # try:
        #     while True:
        #         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
        #         time.sleep(0.4)
        # except Exception as e:
        #     print('finish')

    time.sleep(3)
    html = driver.page_source
    bs = BeautifulSoup(html, 'lxml')
    reviews = bs.select('li.YeINN')

    for r in reviews: # 왜 안돌아가?
        user_id = r.select_one('div.VYGLG')
        content = r.select_one('div.ZZ4OK.IwhtZ')
        try :
            tag = r.select_one('div.gyAGI span.P1zUJ').text
        except :
            tag = ""
        date = r.select('div._7kR3e>span.tzZTd>time')[0]
        revisit = r.select('div._7kR3e>span.tzZTd')[1]

        # 아무것도 없을 때
        user_id = user_id.text if user_id else ''
        content = content.text if content else ''
        tag = tag if tag else ''
        date = date.text if date else ''
        revisit = revisit.text if revisit else ''
        time.sleep(0.06)

        # Append data to lists
        user_id_list.append(user_id)
        content_list.append(content)
        review_category_name_list.append(tag)
        date_list.append(date)
        revisit_list.append(revisit)            

except Exception as e:
    print(e)

finally:
    driver.quit()

# Create a DataFrame
review_data_list = {
    #'매장명': restaurant_name_list,
    'user_id': user_id_list,
    'content': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}



# 태그 데이터 프레임 완성
review_result_df = pd.DataFrame(review_data_list)

review_result_df

# test : 한 번에 tag와 review data를 가져오고 싶은데 마음대로 안됨...

In [ ]:
# 일부 테스트
res_df_test = res_df.copy()
new_res_df_test = res_df_test.iloc[23:26].copy()
new_res_df_test.reset_index(drop=True, inplace=True)
new_res_df_test

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"역삼로 140, 쓰촨 마라",중국식,쓰촨 마라,쓰촨 마라,서울특별시,강남구,"역삼로 140,",서울특별시 강남구 역삼동 834-46 청암빌딩,"서울특별시 강남구 역삼로 140, 청암빌딩 지하2층 2호 (역삼동)",203016.411354,443565.052162,가능,영업/정상
1,"논현로151길 55, 스시 쥬고야",일식,스시 쥬고야,스시 쥬고야,서울특별시,강남구,"논현로151길 55,",서울특별시 강남구 신사동 541-2 호경빌딩,"서울특별시 강남구 논현로151길 55, 호경빌딩 지상4층 (신사동)",202023.586080,446361.308115,가능,영업/정상
2,"역삼로 140, 조선육회",한식,조선육회,조선육회,서울특별시,강남구,"역삼로 140,",서울특별시 강남구 역삼동 834-46 청암빌딩,"서울특별시 강남구 역삼로 140, 청암빌딩 지하2층 6호 (역삼동)",203016.411354,443565.052162,가능,영업/정상


In [ ]:
# 빈 리스트 생성
user_id_list = []
content_list = []
date_list = []  
revisit_list = []
tag_category_name_list = []
restaurant_name_list = []
review_category_name_list = []
price_list = []


# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())

# 해민
driver = webdriver.Chrome()


# 음식점 이름으로 검색하기 
for i in range(0,len(new_res_df_test)):
  
    name = new_res_df_test['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(3)  


    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # '리뷰' 탭의 href 속성 가져와서 리뷰 탭으로 이동하기 
    try : 
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href
        driver.get(review_url)
        time.sleep(3)  

        # 스크롤
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 


        # tag 추출 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

        # 식당 tag를 리스트에 추가
        for item in items:
            restaurant_name_list.append(name)
            tag_category_name_list.append(item.text.split('"')[1])
            price_list.append(int(''.join(filter(str.isdigit, item.text))))

        tag_data_list = {
            '매장명': restaurant_name_list,
            'tag': tag_category_name_list,
            '인원': price_list
        }

    #     # 여기가 안 먹혀요. 수정이 필요합니다. 
    except :
        tag_data_list = {
            '매장명': restaurant_name_list,
            'tag': '리뷰 10개 미만',
            '인원': ''
        }
        continue



# # 리뷰 스크래핑 시작 -> 여기가 합쳐놓으니 안먹혀요
try:
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    # url = 'https://m.place.naver.com/restaurant/1159180795/review/visitor'
    # res = driver.get(url)
    # driver.implicitly_wait(2)

    # 스크롤
    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 

        # '더보기' 클릭
        more_button = driver.find_element(By.XPATH, '//div[@class="lfH3O"]/a[@role="button"]')
        more_button.click()
    
    
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(0.4)

    html = driver.page_source
    bs = BeautifulSoup(html, 'lxml')
    reviews = bs.select('li.YeINN')

    for r in reviews:
        user_id = r.select_one('div.VYGLG')
        content = r.select_one('div.ZZ4OK.IwhtZ')
        tag = r.select_one('div.gyAGI span.P1zUJ').text
        date = r.select('div._7kR3e>span.tzZTd>time')[0]
        revisit = r.select('div._7kR3e>span.tzZTd')[1]

        # 아무것도 없을 때
        user_id = user_id.text if user_id else ''
        content = content.text if content else ''
        tag = tag if tag else ''
        date = date.text if date else ''
        revisit = revisit.text if revisit else ''
        time.sleep(0.06)

        # Append data to lists
        user_id_list.append(user_id)
        content_list.append(content)
        review_category_name_list.append(tag)
        date_list.append(date)
        revisit_list.append(revisit)

# Handle exceptions
except Exception as e:
    print(e)

# # Finally, close the WebDriver
# finally:
#     driver.quit()

# Create a DataFrame
review_data_list = {
    #'매장명': restaurant_name_list,
    'user_id': user_id_list,
    'content': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}

# 태그 데이터 프레임 완성
tag_result_df = pd.DataFrame(tag_data_list)
review_result_df = pd.DataFrame(review_data_list)



Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="lfH3O"]/a[@role="button"]"}
  (Session info: chrome=122.0.6261.57)



In [ ]:
tag_result_df.tail()

,매장명,tag,인원
5,"양재대로33길 24, 브레드_히어_",빵이 맛있어요,13
6,"양재대로33길 24, 브레드_히어_",건강한 맛이에요,6
7,"양재대로33길 24, 브레드_히어_",친절해요,5
8,"양재대로33길 24, 브레드_히어_",매장이 청결해요,5
9,"양재대로33길 24, 브레드_히어_",아늑해요,3


In [ ]:
review_result_df.tail()

,user_id,content,date,revisit,tag


In [ ]:
review_result_df.to_excel('.data/음식점_리뷰데이터.xlsx')
tag_result_df.to_excel('.data/음식점_태그데이터.xlsx')

# 원본

In [ ]:
# 매장명
place = '강남역_서브웨이'

# 엑셀파일 만들기 
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet(place)
list_sheet.append(['nickname', 'content', 'date', 'revisit','tag'])

# 리뷰 스크래핑 시작
try:
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    # res = driver.get(url)
    # driver.implicitly_wait(2)

    for _ in range(5): # 리뷰 더보기 5번만...
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.4)

        # Pagedown
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        try:
            while True:
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                time.sleep(0.4)
        except Exception as e:
            print('finish')

        time.sleep(30)
        html = driver.page_source
        bs = BeautifulSoup(html, 'lxml')
        reviews = bs.select('li.YeINN')

        for r in reviews:
            nickname = r.select_one('div.VYGLG')
            content = r.select_one('div.ZZ4OK.IwhtZ')
            tag = r.select_one('div.gyAGI span.P1zUJ').text
            date = r.select('div._7kR3e>span.tzZTd>time')[0]
            revisit = r.select('div._7kR3e>span.tzZTd')[1]

            # 아무것도 없을 때
            nickname = nickname.text if nickname else ''
            content = content.text if content else ''
            tag = tag if tag else ''
            date = date.text if date else ''
            revisit = revisit.text if revisit else ''
            time.sleep(0.06)

            print(nickname, '/', content, '/', date, '/', revisit, '/',tag)
            list_sheet.append([nickname, content, date, revisit,tag])
            time.sleep(0.06)

        # 저장하기
        file_name = 'naver_review_' + place + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
        xlsx.save(file_name)

# 에러가 나는 경우
except Exception as e:
    print(e)
    # Save the file(temp)
    file_name = 'naver_review_'+ place + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)